In [96]:
#import base libraries
import numpy as np
import pandas as pd
import geopandas as gpd

#import dashboarding libraries
import panel as pn
pn.extension('tabulator')
import geoviews.tile_sources as gvts
import holoviews as hv
import hvplot.pandas
from holoviews import opts
from holoviews.plotting.links import DataLink

In [97]:
#reading in Charitable Ventures data - edited spreadsheet and city boundaries
CVsword = pd.read_csv("data/CV_CITY_CBO_KC_02.27.22.csv", encoding= 'unicode_escape')
cities = gpd.read_file("data/City_Boundaries.geojson")

In [98]:
pd.set_option("display.max_columns",None) #expands number of viewable columns
CVsword = CVsword.fillna(0) #replaces NaNs with 0s to preserve all rows

In [99]:
#define projection as CRS EPSG:6426 for CA zone 6
cities = cities.to_crs(epsg=6426)
cities = cities.drop(['JURISDICTI','Acres','Area_SqMi','OCSurveyDBOCityBoundariesArea'],axis=1) #drop extra columns

In [100]:
#make join columns match
city_data = CVsword.rename(columns={'City':'CITY'})

#make join values match
city_data['CITY'] = city_data['CITY'].str.capitalize()
cities['CITY'] = cities['CITY'].str.capitalize()

In [101]:
#join to city boundaries
city_data_joined = cities.merge(city_data, on='CITY', how='left').fillna(0)

#project
city_data_joined=gpd.GeoDataFrame(city_data_joined, geometry='geometry')
city_data_joined=city_data_joined.to_crs(epsg=6426)

In [102]:
#removing unincorporateds - not sure if i use this later...may be able to remove
city_data_joined_Unincorporateds = city_data_joined.loc[city_data_joined['CITY']=='Unincorporated']
city_data_joined_incorporateds = city_data_joined.loc[city_data_joined['CITY']!='Unincorporated']

city_data_joined_incorporateds = city_data_joined_incorporateds.drop_duplicates(subset=['CITY'], keep='first')
city_data_joined = pd.concat([city_data_joined_incorporateds,city_data_joined_Unincorporateds])

In [103]:
#read in cbo data in long form
CBO_data = pd.read_csv("data/CV_CITY_CBO_KC_2022feb27_long.csv", encoding = 'unicode_escape')
#drop unwanted columns by name
CBO_data = CBO_data.drop(['cbo_string','cbo_count'], axis=1)
#drop first unnamed column
CBO_data = CBO_data.iloc[:,1:]
#group by city column
CBOcount_byCity = CBO_data.groupby(['City']).count().reset_index()
#make join columns and values match
CBOcount_byCity = CBOcount_byCity.rename(columns={'City':'CITY'})
CBOcount_byCity['CITY'] = CBOcount_byCity['CITY'].str.capitalize()
#add column of cbo list
CBOcount_byCity = CBOcount_byCity.merge(city_data, on='CITY').rename(columns={'cbo_name_x':'cbo_count', 'cbo_name_y':'cbo_names'})
#join to geometries
CBOcount_geo = cities.merge(CBOcount_byCity, on='CITY', how='left').fillna(0)
#drop unincorporated territories
CBOcount_geo = CBOcount_geo.loc[CBOcount_geo['CITY']!='Unincorporated']

In [104]:
#make table
citiesList = CBOcount_geo['CITY'].to_numpy()
cboList = CBOcount_geo['cbo_names'].to_numpy()

table = hv.Table({'City':citiesList, 'CBO':cboList},
                ['City','CBO'])

In [105]:
#make map
map00 = CBOcount_geo.hvplot(
c='cbo_count',
frame_width=600,
frame_height=500,
dynamic=True,
geo=True,
crs=6426,
hover_cols=['CITY'],
cmap='BuGn',
title='CBOs by City')

basemap = gvts.CartoLight.opts(alpha=0.6)

map01=basemap*map00

In [106]:
#combine into dashboard
DataLink(map00, table)

dash=(table+map00*basemap).opts(
    opts.Table(width=500),
    opts.Polygons(width=500, height=500, tools=['hover','tap'], xaxis=None,
                 yaxis=None))

In [107]:
#data processing for org table
CVactivities= pd.read_csv("data/CVActivities_KC_02.26.22.csv", encoding= 'unicode_escape')
CVactivities = CVactivities.fillna("No Data")

In [108]:
CVactivities = pd.DataFrame(CVactivities)
CVactivities

,cbo_name,activities,City,HTC,Language
0,Access California Services,"Social Media post, Friday Prayer, distributed ...",Anaheim,Born Outside of U.S.,"Arabic, Hindi"
1,CAIR-LA,"event, flyers",Anaheim,"Born Outside of U.S., Limited English Speaking...","Arabic, Hindi, Farsi, Other"
2,CAP OC,"booths, collateral, flyers, phone_banking",Anaheim,Children Under 5,No Data
3,Clergy and Laity United for Economic Justice,No Data,Anaheim,No Data,No Data
4,Institute for Healthcare Advancement,No Data,Anaheim,No Data,No Data
...,...,...,...,...,...
131,MOMS Orange County,education_forum,Westminster,No Data,"Spanish, Vietnamese"
132,OCAPICA,press conference,Westminster,Born Outside of U.S.,No Data
133,Orange County United Way,event,Westminster,"Born Outside of U.S., Children Under 5, Limite...","Farsi, russian, Spanish, Vietnamese, Other"
134,"Southland Integrated Services, Inc.",food distribution at community center,Westminster,"Born Outside of U.S., Limited English Speaking...",Vietnamese


In [109]:
filter_table=pn.widgets.Tabulator(CVactivities, 
                                  layout='fit_columns', 
                                  width=800, 
                                  embed_content=True)

In [110]:
filter_table

Tabulator(layout='fit_columns', value=                          ..., width=800)

In [111]:
#create city widget
cityNames_list = city_data_joined['CITY'].unique()
cityNames_arr = np.asarray(cityNames_list).astype('str')
cityNames = sorted(np.char.title(cityNames_arr).astype('O').tolist())

citySelect=pn.widgets.MultiSelect(options=cityNames, name='City Filter', height=300, )
filter_table.add_filter(citySelect, 'City')

#make a clear button
#clearButton = button(label="Clear")
#callback = CustomJS(args=dict(s=select), code="s.value=[]")
#clearButton.js_on_event('button_click',callback)

#create file name box and download button
filename, button = filter_table.download_menu(
    text_kwargs={'name':'Enter filename', 'value':'OC_2020_Census_Activities_SHIELD.csv'},
    button_kwargs={'name':'Download table'}
)

In [112]:
#create language widget
CVactivities_long = pd.read_csv("data/CVActivities_KC_2022feb22_long.csv")
language_list = CVactivities_long['language'].unique()
language_arr = np.asarray(language_list).astype('str')
languages = sorted(np.char.title(language_arr).astype('O').tolist())

langSelect=pn.widgets.MultiSelect(options=languages, name='Language Filter', height=300)
filter_table.add_filter(langSelect, 'Language')

In [113]:
#trying type widgets
city_filter = pn.widgets.TextInput(name='City Search', value='')
HTC_filter = pn.widgets.TextInput(name='HTC Search', value='')
lang_filter = pn.widgets.TextInput(name='Language Search', value='')

def contains_filter(df, pattern, column):
    if not pattern:
        return df
    return df[df[column].str.contains(pattern, na=False)]
    
filter_table.add_filter(pn.bind(contains_filter, pattern=city_filter, column='City'))
filter_table.add_filter(pn.bind(contains_filter, pattern=HTC_filter, column='HTC'))
filter_table.add_filter(pn.bind(contains_filter, pattern=lang_filter, column='Language'))

#pn.Row(city_filter, HTC_filter, lang_filter, filter_table)

# Orange County Census + Network Directory

### Hosted by [Charitable Ventures of Orange County](https://charitableventuresoc.org/) | [Good Work Collaborative](https://www.ourgoodwork.co/) (2022)

### About

This map and table directory show which organizations reported outreach activities in Orange County for the 2020 Census. The information is organized and presented by city, outreach language, and demographic focus (i.e. “hard-to-count” (HTC) characteristic). The darker the color of the city on the map, the more organizations that reported outreach activities. For example, according to the map, more organizations reported census outreach activities in Santa Ana (22) than in Newport Beach (1).

### Instructions:

- <b>Simple Search or Advanced Search:</b>
    - <b>Simple Search:</b> Click any city on the map to see how many organizations reported outreach activities for the 2020 Census in that city. A list of those organizations will appear in the table below, along with details about their reported activities.
    - <b>Advanced Search:</b> Enter the city name(s), language(s), and/or “hard-to-count characteristic(s)” into the appropriate search fields below, and then press “enter” on your keyboard. Note that the fields are case sensitive and names must be typed exactly as they appear (e.g., “Anaheim”, not “anaheim”).
- <b>Sort Table:</b> Click the header of the column in order to alphabetize that column. For example, click “City” to sort the list of cities in alphabetical order.
- <b>Download Table:</b> Enter a filename relevant to your search (e.g. Spanish outreach in Anaheim) and click “Download Table”, which will save a spreadsheet file in the computer’s ‘Downloads’ or ‘Desktop’ folder.
- <b>Reset Search:</b> Delete any words in the search fields and press “enter” on your keyboard to reset your search.

In [114]:
with open('README.md','r') as file:
    narrative = file.read()
    print(narrative)
    file.close()

# Orange County Census + Network Directory<br>
### Hosted by [Charitable Ventures of Orange County](https://charitableventuresoc.org/) and [Good Work Collaborative](https://www.ourgoodwork.co/) (2022)<br>
### About<br>
This map and table directory show which organizations reported outreach activities in Orange County for the 2020 Census. The information is organized and presented by city, outreach language, and demographic focus (i.e. "hard-to-count" (HTC) characteristic). The darker the color of the city on the map, the more organizations that reported outreach activities. For example, according to the map, more organizations reported census outreach activities in Santa Ana (22) than in Newport Beach (1).<br>
### Instructions:<br>
- <b>Simple Search or Advanced Search:</b>
	- <b>Simple Search:</b> Click any city on the map to see how many organizations reported outreach activities for the 2020 Census in that city. A list of those organizations will appear in the table below, along wit

In [115]:
pn.extension()

dash3 = pn.Row(map01,
               pn.Column(city_filter,
                         HTC_filter,
                         lang_filter,
                         filename, 
                         button,
                         filter_table)).servable()

dash3.servable()

Row
    [0] HoloViews(Overlay)
    [1] Column
        [0] TextInput(name='City Search')
        [1] TextInput(name='HTC Search')
        [2] TextInput(name='Language Search')
        [3] TextInput(name='Filename', value='table.csv')
        [4] Button(name='Download table')
        [5] Tabulator(layout='fit_columns', value=                          ..., width=800)